In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

NameError: name 'pd' is not defined

In [4]:
#Let's try BeautifulSoup to scrape OSHA penalties of more than $40,000.
response = requests.get("https://www.osha.gov/enforcement/toppenalties/bystate")
response_html = response.text
data_bso = BeautifulSoup(response_html)

In [5]:
print(data_bso)

<html><head>
<title>Access Denied</title>
</head><body>
<h1>Access Denied</h1>
 
You don't have permission to access "http://www.osha.gov/enforcement/toppenalties/bystate" on this server.<p>
Reference #18.42d3e17.1690248825.2f03778e
</p></body>
</html>



In [6]:
#That didn't work. I'll try to find an undocumented API, instead.

In [7]:
baseurl = "https://www.osha.gov/sites/default/files/json/enforcement-cases/data-set.json"

In [8]:
response = requests.get(baseurl)

In [9]:
response.json()
#Oh, weird.
#Let's try cURLing it next.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
#Got this from the cURL converter.
import requests

cookies = {
    '_ga': 'GA1.2.1221182387.1689314175',
    '_gat_GSA_ENOR0': '1',
    '_gat_GSA_ENOR1': '1',
    '_gid': 'GA1.2.1390347906.1689979681',
}

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    # 'Cookie': '_ga=GA1.2.1221182387.1689314175; _gat_GSA_ENOR0=1; _gat_GSA_ENOR1=1; _gid=GA1.2.1390347906.1689979681',
    'Accept-Language': 'en-US,en;q=0.9',
    'If-None-Match': '"64bb0974-247ec5"',
    'Host': 'www.osha.gov',
    'If-Modified-Since': 'Fri, 21 Jul 2023 22:40:52 GMT',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'Referer': 'https://www.osha.gov/enforcement/toppenalties/bystate',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'X-Requested-With': 'XMLHttpRequest',
}

response = requests.get(
    'https://www.osha.gov/sites/default/files/json/enforcement-cases/data-set.json',
    cookies=cookies,
    headers=headers,
)

In [11]:
#Ohhhh, fuck, I forgot to import JSON. That's why it didn't work.
import json
data = response.json()

In [12]:
#I found that the dataset is available as a table here. Let's see if it works better:
requests.get("https://www.osha.gov/ords/imis/establishment.inspection_detail?id=%201654638.015")

<Response [403]>

In [13]:
#It doesn't.

In [14]:
#I couldn't find any other undocumented API pages. We'll work with this one.
#Let's find what the top 5 most—

#Shit, I just realized that the table here does the work for us, and my insights are limited because
#get requests are restricted. https://www.osha.gov/enforcement/toppenalties/

In [15]:
#Let's pivot — what can I find that OSHA's table doesn't easily let me find?
#How about: What companies have the most penalties of over $40,000?
#How much do these penalties add up to?
#What cities have the most penalties of over $40,000?
#And how much do their penalties of $40,000+ add up to?

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(data)

In [18]:
df['employer'].value_counts().head(20)

employer
Dollar Tree Stores, Inc.                       72
U.S. Postal Service                            44
Dollar General Corporation/ Dolgencorp, LLC    27
Dollar General Corporation                     27
U.S. POSTAL SERVICE                            22
Dolgencorp, LLC                                16
Kaiser Foundation Hospitals                    12
Target Corporation                             11
United States Postal Service                   11
Dollar General                                 10
The Goodyear Tire & Rubber Company             10
AMERICA 1ST ROOFING & BUILDERS INC             10
Great White Construction, Inc.                  9
YOUNG CORP                                      9
ALLWAYS ROOFING INC.                            8
Berlin Builders, Inc.                           8
AJM Packaging Corporation                       8
D.R. Horton, Inc.                               8
Pilgrim's Pride Corporation                     8
All Best Contractors Corporation         

In [19]:
#This is too easy. Let's get the rate of increase for the top 10 organizations per year.

In [20]:
df.head()

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current


In [21]:
import re

In [22]:
df['state'].value_counts()

state
CA    1614
OH    1150
NJ     929
TX     927
IL     689
FL     484
GA     480
NY     474
PA     406
MA     371
WA     369
WI     359
MO     288
AL     232
VA     217
KS     172
CT     168
MI     166
NV     141
IA     140
NC     133
AR     127
DE     112
CO     109
KY     108
OK     102
AK      98
NH      93
MS      80
MN      80
LA      78
HI      78
TN      76
MD      74
NE      71
ME      69
WV      65
RI      61
IN      54
ND      52
OR      51
ID      45
PR      36
SD      36
VT      33
NM      29
WY      28
AZ      21
MT      18
DC      16
SC      16
VI      11
GU       5
UT       5
AS       5
MP       5
Name: count, dtype: int64

In [23]:
def get_year(date):
    pattern = r"^\d\d/\d\d/(\d{4})"
    replacement = r"\1"
    return int(re.sub(pattern, replacement, date))
print(get_year('12/30/2023'))

2023


In [24]:
df['issuance_date'].str.replace(r"^\d\d/\d\d/(\d{4})", r"\1", regex=True)

0        2023
1        2023
2        2023
3        2023
4        2023
         ... 
11851    2015
11852    2015
11853    2015
11854    2015
11855    2019
Name: issuance_date, Length: 11856, dtype: object

In [25]:
#I DID MY FIRST REGEX!!!

In [26]:
df['Year'] = df['issuance_date'].str.replace(r"^\d\d/\d\d/(\d{4})", r"\1", regex=True).astype(int)

In [27]:
df.head()

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current,2023
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current,2023
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current,2023
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current,2023
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current,2023


In [28]:
#I thought about it more.
#Let's look at the top companies and see how their number of penalties per year changed,
#along with the additional penalties they accrued per year.

In [29]:
df.query('Year == 2023').head()

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current,2023
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current,2023
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current,2023
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current,2023
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current,2023


In [30]:
df[df.Year == 2023].head()

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current,2023
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current,2023
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current,2023
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current,2023
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current,2023


In [31]:
#Here's what the process will be:
#1. Look at the top 20 companies and merge companies with the same name. (Ultimately, I want
# to be able to get the top 10 companies, accounting for merged names.)
#
#2. Compare the value counts per year for each business.

In [33]:
#We'll manually go through this:
df['employer'].value_counts().head(10)

employer
Dollar Tree Stores, Inc.                       72
U.S. Postal Service                            44
Dollar General Corporation/ Dolgencorp, LLC    27
Dollar General Corporation                     27
U.S. POSTAL SERVICE                            22
Dolgencorp, LLC                                16
Kaiser Foundation Hospitals                    12
Target Corporation                             11
United States Postal Service                   11
Dollar General                                 10
Name: count, dtype: int64

In [34]:
employers_list_fixed = []

for cell in df['employer']:
    if "Dollar Tree" in cell.strip().title() or "Family Dollar" in cell.strip().title():
        cell_dt = cell.replace(cell, "Dollar Tree/Family Dollar (similar names merged)")
        employers_list_fixed.append(cell_dt)
    elif "Dollar General" in cell.strip().title() or "Dolgencorp" in cell.strip().title():
        cell_dg = cell.replace(cell, "Dollar General (similar names merged)")
        employers_list_fixed.append(cell_dg)
    elif "Postal Service" in cell.strip().title():
        cell_ps = cell.replace(cell, "U.S. Postal Service (similar names merged)")
        employers_list_fixed.append(cell_ps)
    elif "Target" in cell.strip().title():
        cell_tgt = cell.replace(cell, "Target (similar names merged)")
        employers_list_fixed.append(cell_tgt)
    elif "Kaiser" in cell.strip().title():
        cell_ksr = cell.replace(cell, "Kaiser (similar names merged)")
        employers_list_fixed.append(cell_ksr)
    else:
        employers_list_fixed.append(cell)

In [35]:
df['employers_fixed'] = employers_list_fixed

In [36]:
df.head(5)

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year,employers_fixed
0,11030,01/25/2023,KING COVE,AK,"Peter Pan Seafood Company, LLC",1612177.015,"$130,518.00",Current,2023,"Peter Pan Seafood Company, LLC"
1,11097,02/24/2023,FORT WAINWRIGHT,AK,"Bering Straits Technical Services, LLC",1618057.015,"$45,680.00",Current,2023,"Bering Straits Technical Services, LLC"
2,11231,03/09/2023,AKUTAN,AK,Trident Seafoods Corporation,1621386.015,"$45,506.00",Current,2023,Trident Seafoods Corporation
3,11642,05/30/2023,CORDOVA,AK,Prince William Sound Aquaculture Corporation,1637084.015,"$87,012.00",Current,2023,Prince William Sound Aquaculture Corporation
4,11715,06/05/2023,PALMER,AK,Matanuska-Susitna Borough School District,1641670.015,"$111,875.00",Current,2023,Matanuska-Susitna Borough School District


In [115]:
#Let's make this the first chart to give readers an overview of the most fined employers.
df['employers_fixed'].value_counts().head(10)

df_bargraph = pd.DataFrame({
    'Count': df['employers_fixed'].value_counts().head(10)
}).reset_index()

In [117]:
df_bargraph

,employers_fixed,Count
0,Dollar Tree/Family Dollar (similar names merged),147
1,U.S. Postal Service (similar names merged),126
2,Dollar General (similar names merged),124
3,Target (similar names merged),19
4,Kaiser (similar names merged),16
5,AMERICA 1ST ROOFING & BUILDERS INC,10
6,The Goodyear Tire & Rubber Company,10
7,YOUNG CORP,9
8,"Great White Construction, Inc.",9
9,"Berlin Builders, Inc.",8


In [38]:
#I'm going to manually check to ensure that I didn't merge any name incorrectly.

In [39]:
df[df.employers_fixed == "Dollar Tree/Family Dollar (similar names merged)"]

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year,employers_fixed
146,3579,11/29/2019,ANDALUSIA,AL,"Dollar Trees Stores, Inc.",1406101.015,"$312,576.00",Current,2019,Dollar Tree/Family Dollar (similar names merged)
148,3622,12/06/2019,RAINBOW CITY,AL,"Dollar Tree Stores, Inc.",1411428.015,"$104,192.00",Current,2019,Dollar Tree/Family Dollar (similar names merged)
160,3809,03/19/2020,FOLEY,AL,"Dollar Tree Stores, Inc.",1454236.015,"$106,029.00",Current,2020,Dollar Tree/Family Dollar (similar names merged)
257,10895,01/26/2023,FLORENCE,AL,"Dollar Tree, Inc.",1612521.015,"$171,886.00",Current,2023,Dollar Tree/Family Dollar (similar names merged)
365,3862,04/08/2020,WEST MEMPHIS,AR,Family Dollar Distribution Center,1437095.015,"$86,941.00",Current,2020,Dollar Tree/Family Dollar (similar names merged)
...,...,...,...,...,...,...,...,...,...,...
11405,10210,08/26/2022,GRAFTON,WI,"Dollar Tree Stores, Inc.",1586794.015,"$91,157.00",Current,2022,Dollar Tree/Family Dollar (similar names merged)
11504,8774,08/25/2021,GRAFTON,WI,"Dollar Tree Stores, Inc.",1529043.015,"$93,117.00",Current,2021,Dollar Tree/Family Dollar (similar names merged)
11655,3847,03/19/2020,MARINETTE,WI,"Dollar Tree Stores, Inc.",1433182.015,"$477,089.00",Current,2020,Dollar Tree/Family Dollar (similar names merged)
11694,11682,06/02/2023,PEWAUKEE,WI,DOLLAR TREE STORES INC.,1642112.015,"$98,219.00",Current,2023,Dollar Tree/Family Dollar (similar names merged)


In [40]:
df[df.employers_fixed == "Dollar General (similar names merged)"]

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year,employers_fixed
99,9412,02/23/2022,MOBILE,AL,"Dolgencorp, LLC",1550484.015,"$273,472.00",Current,2022,Dollar General (similar names merged)
100,9413,02/23/2022,MOBILE,AL,"Dolgencorp, LLC",1550619.015,"$273,472.00",Current,2022,Dollar General (similar names merged)
101,9414,02/23/2022,MOBILE,AL,"Dolgencorp, LLC",1550627.015,"$136,736.00",Current,2022,Dollar General (similar names merged)
139,11398,04/27/2023,ADDISON,AL,"Dollar General Corporation/ Dolgencorp, LLC",1631749.015,"$450,927.00",Current,2023,Dollar General (similar names merged)
141,11487,05/08/2023,HALEYVILLE,AL,"Dollar General Corporation/ Dolgencorp, LLC",1633786.015,"$450,927.00",Current,2023,Dollar General (similar names merged)
...,...,...,...,...,...,...,...,...,...,...
11686,11397,04/11/2023,OCONTO FALLS,WI,"Dolgencorp, LLC",1636766.015,"$257,829.00",Current,2023,Dollar General (similar names merged)
11689,11579,05/15/2023,KEWAUNEE,WI,"DolgenCorp, LLC",1634517.015,"$367,216.00",Current,2023,Dollar General (similar names merged)
11720,9828,05/26/2022,BALDWIN,WI,"Dolgencorp, LLC",1566292.015,"$435,081.00",Current,2022,Dollar General (similar names merged)
11809,2511,09/21/2015,MADISON,WV,Dollar General Corporation,1048760.015,"$103,000.00",Current,2015,Dollar General (similar names merged)


In [41]:
df[df.employers_fixed == "U.S. Postal Service (similar names merged)"]

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year,employers_fixed
166,3979,05/14/2020,MADISON,AL,U. S. Postal Service,1451114.015,"$106,029.00",Current,2020,U.S. Postal Service (similar names merged)
188,4254,03/15/2017,PHENIX CITY,AL,U.S. Postal Service,1184859.015,"$57,940.00",Current,2017,U.S. Postal Service (similar names merged)
211,524,07/12/2018,SYCAMORE,AL,U.S. Postal Service,1303930.015,"$50,815.00",Current,2018,U.S. Postal Service (similar names merged)
272,4271,02/01/2019,BIRMINGHAM,AL,U.S. Postal Service,1345663.015,"$62,513.00",Current,2019,U.S. Postal Service (similar names merged)
287,4286,02/05/2020,BIRMINGHAM,AL,U.S. Postal Service,1422749.015,"$137,830.00",Current,2020,U.S. Postal Service (similar names merged)
...,...,...,...,...,...,...,...,...,...,...
10982,1675,03/16/2017,CHESAPEAKE,VA,U.S. POSTAL SERVICE,1194247.015,"$117,699.00",Current,2017,U.S. Postal Service (similar names merged)
11783,7819,01/19/2017,PARKERSBURG,WV,United States Postal Service,1169367.015,"$69,713.00",Current,2017,U.S. Postal Service (similar names merged)
11788,7824,11/15/2017,KENOVA,WV,U.S. Postal Service Kenova,1234748.015,"$63,375.00",Current,2017,U.S. Postal Service (similar names merged)
11789,7825,01/19/2018,PARKERSBURG,WV,U.S. Postal Service,1249372.015,"$71,137.00",Current,2018,U.S. Postal Service (similar names merged)


In [42]:
df[df.employers_fixed == "Target (similar names merged)"]

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year,employers_fixed
71,10636,11/28/2022,WASILLA,AK,"Target Stores, Inc.",1601188.015,"$62,151.00",Current,2022,Target (similar names merged)
327,9130,11/23/2021,TUSCALOOSA,AL,Target Corporation,1533241.015,"$40,959.00",Current,2021,Target (similar names merged)
386,11431,05/01/2023,JONESBORO,AR,Target Corporation,1633197.015,"$147,334.00",Current,2023,Target (similar names merged)
1647,7889,11/18/2020,FONTANA,CA,Target Corporation,1479936.015,"$50,000.00",Current,2020,Target (similar names merged)
2225,3419,09/23/2019,WATERFORD,CT,Target Corporation,1413674.015,"$83,763.00",Current,2019,Target (similar names merged)
2313,5050,06/14/2019,STAMFORD,CT,Target Corporation,1371131.015,"$168,537.00",Current,2019,Target (similar names merged)
4840,3660,12/31/2019,FRAMINGHAM,MA,Target Corporation,1414036.015,"$113,652.00",Current,2019,Target (similar names merged)
4841,3661,12/31/2019,DANVERS,MA,Target Corporation,1422713.015,"$121,229.00",Current,2019,Target (similar names merged)
4990,5843,05/21/2019,EVERETT,MA,Target Corporation,1381150.015,"$117,452.00",Current,2019,Target (similar names merged)
5320,3694,01/16/2020,BANGOR,ME,Target Corporation,1416498.015,"$95,420.00",Current,2020,Target (similar names merged)


In [43]:
df[df.employers_fixed == "Kaiser (similar names merged)"]

,id,issuance_date,city,state,employer,inspection_number,initial_penalty,status,Year,employers_fixed
596,4185,11/06/2020,OAKLAND,CA,Kaiser Foundation Hospitals/Kaiser Permanente ...,1474630.015,"$78,300.00",Current,2020,Kaiser (similar names merged)
624,8592,05/03/2021,SAN RAFAEL,CA,Kaiser Foundation Hospitals,1486133.015,"$56,120.00",Current,2021,Kaiser (similar names merged)
663,8887,09/24/2021,DOWNEY,CA,Kaiser Foundation Hospitals,1511445.015,"$195,500.00",Current,2021,Kaiser (similar names merged)
665,8889,08/02/2021,BAKERSFIELD,CA,Kaiser Foundation Hospitals,1491759.015,"$88,885.00",Current,2021,Kaiser (similar names merged)
794,2400,12/03/2015,VALLEJO,CA,Kaiser Foundation Hospitals,1068406.015,"$149,900.00",Current,2015,Kaiser (similar names merged)
898,9532,09/29/2021,ROSEVILLE,CA,Kaiser Foundation Hospitals,1490747.015,"$57,385.00",Current,2021,Kaiser (similar names merged)
1625,5002,11/05/2020,SANTA ROSA,CA,Kaiser Foundation Hospitals,1474648.015,"$55,350.00",Current,2020,Kaiser (similar names merged)
1628,5005,11/23/2020,SAN JOSE,CA,Kaiser Foundation Hospital,1476073.015,"$85,375.00",Current,2020,Kaiser (similar names merged)
1644,7886,12/28/2020,ANTIOCH,CA,Kaiser Permanente Antioch Medical Center,1475603.015,"$56,000.00",Current,2020,Kaiser (similar names merged)
1653,7945,01/04/2021,SAN LEANDRO,CA,Kaiser Foundation Hospitals,1479549.015,"$87,950.00",Current,2021,Kaiser (similar names merged)


In [44]:
#Cool, the "Kaiser Aluminum & Fabricated Products, LLC" clearly doesn't belong here!
#Let's revert its employers_fixed name.

df.loc[10503]['employers_fixed'].replace('Kaiser (similar names merged)', 'Kaiser Aluminum & Fabricated Products, LLC')

'Kaiser Aluminum & Fabricated Products, LLC'

In [45]:
#I tried str.replace, regex=True and inplace=True as well as redefining the cell, but none of it worked.

In [46]:
#How about this:
df.loc[10503, 'employers_fixed'] = 'Kaiser Aluminum & Fabricated Products, LLC'

In [47]:
df.loc[10503]['employers_fixed']

'Kaiser Aluminum & Fabricated Products, LLC'

In [120]:
df_bargraph.to_csv('lede_project-3_df-for-bargraph.csv', index=False)

In [48]:
#Cool. Now, I should prototype the charts I want to make for the top five employers.

In [49]:
#Perhaps make a chart from the following gross amounts owed.
#Let's also get the number of employees per employer to compare dollars owed per employee. 
#This should actually be the middle chart.

#How much has dollar tree owed in total from penalties of $40,000 or more?
df[df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)']['initial_penalty'].str.strip().str.replace(',', '').str.replace('$', '').astype(float).sum()

27508719.0

In [50]:
#How about Target?
df[df.employers_fixed == 'Target (similar names merged)']['initial_penalty'].str.strip().str.replace(',', '').str.replace('$', '').astype(float).sum()

1624413.0

In [51]:
df[df.employers_fixed == 'Kaiser (similar names merged)']['initial_penalty'].str.strip().str.replace(',', '').str.replace('$', '').astype(float).sum()

1249885.0

In [52]:
df[df.employers_fixed == 'Dollar General (similar names merged)']['initial_penalty'].str.strip().str.replace(',', '').str.replace('$', '').astype(float).sum()

24653949.0

In [53]:
df[df.employers_fixed == 'U.S. Postal Service (similar names merged)']['initial_penalty'].str.strip().str.replace(',', '').str.replace('$', '').astype(float).sum()

11778822.0

In [54]:
pd.DataFrame([{
    'Dollar Tree/Family Dollar': 27508719,
    'Dollar General': 24653949,
    'U.S. Postal Service': 11778822,
    'Target': 1624413,
    'Kaiser': 1249885
}]).transpose()

#Next, add a column with the number of employees and a column with the dollars owed per employee.


,0
Dollar Tree/Family Dollar,27508719
Dollar General,24653949
U.S. Postal Service,11778822
Target,1624413
Kaiser,1249885


In [55]:
#You know what? We should just get all of them at once.

In [56]:
#First, let's make a cleaned $ column.
df['initial_penalty_cleaned'] = df['initial_penalty'].str.strip().str.replace(',', '').str.replace('$', '').astype(float)

In [134]:
#Let's use groupby to calculate the sum of the penalties of $40,000+ for each employer.
#We can add this as a chart at the beginning to show that Dollar Tree/Family Dollar also topped the list
# of amount owed
#amount accrued in penalties of $40,000+.
table_amounts = df.groupby('employers_fixed')['initial_penalty_cleaned'].sum().sort_values(ascending=False)
df.groupby('employers_fixed')['initial_penalty_cleaned'].sum().sort_values(ascending=False)

#Perhaps make the top 10 here into a chart to give an overview.

employers_fixed
Dollar Tree/Family Dollar (similar names merged)    27508719.0
Dollar General (similar names merged)               24653949.0
U.S. Postal Service (similar names merged)          11778822.0
Sunfield, Inc.                                       3426900.0
Miracapo Pizza Company LLC dba Little Lady Foods     3102849.0
                                                       ...    
Carlex Glass America, LLC                              40000.0
Powell Electrical Systems, Inc.                        40000.0
L.&G. Construction Company, Inc.                       40000.0
CMCC, LLC                                              40000.0
TESORO REFINING & MARKETING COMPANY LLC                40000.0
Name: initial_penalty_cleaned, Length: 9832, dtype: float64

In [136]:
#Let's create a DF with the dollar amounts and merge it with the DF with the penalty count for a bar graph with
#the dollar amount as a popup.
df_amounts = pd.DataFrame(
    table_amounts
).reset_index()
df_amounts

,employers_fixed,initial_penalty_cleaned
0,Dollar Tree/Family Dollar (similar names merged),27508719.0
1,Dollar General (similar names merged),24653949.0
2,U.S. Postal Service (similar names merged),11778822.0
3,"Sunfield, Inc.",3426900.0
4,Miracapo Pizza Company LLC dba Little Lady Foods,3102849.0
...,...,...
9827,"Carlex Glass America, LLC",40000.0
9828,"Powell Electrical Systems, Inc.",40000.0
9829,"L.&G. Construction Company, Inc.",40000.0
9830,"CMCC, LLC",40000.0


In [138]:
#Great, now let's merge 'em.
df_bargraph_merged = df_bargraph.merge(df_amounts,
               left_on='employers_fixed',
               right_on='employers_fixed',
               how='left')

In [140]:
df_bargraph_merged.to_csv('lede_project-3_df-for-bargraph.csv', index=False)

In [58]:
df['initial_penalty_cleaned'][df.Year == 2023][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

4967323.0

In [59]:
df['initial_penalty_cleaned'][df.Year == 2022][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

4740048.0

In [60]:
df['initial_penalty_cleaned'][df.Year == 2021][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

1486759.0

In [61]:
df['initial_penalty_cleaned'][df.Year == 2020][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

7348248.0

In [62]:
df['initial_penalty_cleaned'][df.Year == 2019][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

4388710.0

In [63]:
df['initial_penalty_cleaned'][df.Year == 2018][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

2197933.0

In [64]:
df['initial_penalty_cleaned'][df.Year == 2017][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

585766.0

In [65]:
df['initial_penalty_cleaned'][df.Year == 2016][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

487832.0

In [66]:
df['initial_penalty_cleaned'][df.Year == 2015][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

1306100.0

In [67]:
df['initial_penalty_cleaned'][df.Year == 2014][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].sum()

0.0

In [68]:
#Now let's get the penalties of $40,000+ per year.

In [69]:
df[df.Year == 2015][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].value_counts()

/var/folders/cl/myc9mwnx15nbmsd02f_ct27h0000gn/T/ipykernel_30764/1104354340.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.Year == 2015][df.employers_fixed == 'Dollar Tree/Family Dollar (similar names merged)'].value_counts()


id    issuance_date  city                   state  employer                                   inspection_number  initial_penalty  status   Year  employers_fixed                                   initial_penalty_cleaned
2490  09/30/2015     AMITYVILLE             NY     Dollar Tree Stores, Inc.                   1060953.015        $140,000.00      Current  2015  Dollar Tree/Family Dollar (similar names merged)  140000.0                   1
2580  08/04/2015     BRIDGEPORT             CT     Family Dollar Stores of Connecticut, Inc.  1066819.015        $72,000.00       Current  2015  Dollar Tree/Family Dollar (similar names merged)  72000.0                    1
2708  05/15/2015     CHICAGO RIDGE          IL     Dollar Tree Stores Inc.                    1009303.015        $77,000.00       Current  2015  Dollar Tree/Family Dollar (similar names merged)  77000.0                    1
2709  05/15/2015     SOUTH CHICAGO HEIGHTS  IL     Dollar Tree Stores Inc.                    1012127.015    

In [70]:
#Whoops — let's try to fix that.

In [71]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2015))])

13

In [72]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2016))])

5

In [73]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2017))])

5

In [74]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2018))])

17

In [75]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2019))])

25

In [76]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2020))])

28

In [77]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2021))])

9

In [78]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2022))])

21

In [79]:
len(df.loc[((df["employers_fixed"] == 'Dollar Tree/Family Dollar (similar names merged)') & (df["Year"] == 2023))])

24

In [80]:
#Let's make a line chart with those numbers.
#Divide by the number of employees.
#Yes, some employees will be FT and some will be PT — but I believe that it's more accurate than
#using the number of stores, given that stores greatly vary in sizes. For example, Dollar General appears to have
#far more stores than Dollar Tree and Target but has fewer employees than either.

In [81]:
#This chart focuses on Dollar Tree. It'll be the second chart.
#I multiplied the original penalty value for 2023, 24, by 12 and divided by 7 since it's July to estimate
#the penalty value at the end of 2023.
#I did the same for the dollars value for 2023 — multiplied 4967323 by 12 and divided by 7 to estimate
#its year-end value.

df_dt = pd.DataFrame({
    'Year': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'Dollars fined (from penalties of $40,000+)': [1306100, 487832, 585766, 2197933, 4388710, 7348248, 1486759, 4740048, 8515410],
    'Penalties' : [13, 5, 5, 17, 25, 28, 9, 21, 41]
})

In [82]:
df_dt




,Year,"Dollars fined (from penalties of $40,000+)",Penalties
0,2015,1306100,13
1,2016,487832,5
2,2017,585766,5
3,2018,2197933,17
4,2019,4388710,25
5,2020,7348248,28
6,2021,1486759,9
7,2022,4740048,21
8,2023,8515410,41


In [83]:
#Let's check Dollar General.
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2015))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2016))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2017))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2018))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2019))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2020))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2021))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2022))]))
print(len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2023))]))

7
11
7
5
2
6
9
29
48


In [84]:
#OMG THAT'S FUCKED UP. THIS HAS TO BE THE FOCUS.

In [85]:
#Here's Dollar General's dollar penalties accrued annually:
print(df['initial_penalty_cleaned'][df.Year == 2015][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2016][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2017][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2018][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2019][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2020][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2021][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2022][df.employers_fixed == 'Dollar General (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2023][df.employers_fixed == 'Dollar General (similar names merged)'].sum())

706350.0
1170926.0
673960.0
396886.0
225417.0
728671.0
1216165.0
8558719.0
10976855.0


In [86]:
#I did some research. Dollar General has been written about, but I don't think that this data
#has been presented like this — that is, that Dollar General has been violating OSHA far more times
#over the last couple of years.

In [87]:
#Let's check Target real quick:

In [88]:
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2015))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2016))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2017))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2018))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2019))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2020))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2021))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2022))]))
print(len(df.loc[((df["employers_fixed"] == 'Target (similar names merged)') & (df["Year"] == 2023))]))

1
1
0
0
9
4
1
2
1


In [89]:
#OK, there's nothing out of the ordinary with Target. Dollar General is by far the most fucked among itself, Target
#and Dollar Tree/Family Dollar.
#Let's create a DF for Dollar General where the x axis is the year, and the y axis is penalty-number-per-TK employees.
#I pulled employee count data from the company's 10k reports.

#Let's also get the estimated year-end for 2023 (I multiplied the final dollar amount and penalty by 12/7 since 
#we're in July):

print((df['initial_penalty_cleaned'][df.Year == 2023][df.employers_fixed == 'Dollar General (similar names merged)'].sum()) * 12 / 7)
print((len(df.loc[((df["employers_fixed"] == 'Dollar General (similar names merged)') & (df["Year"] == 2023))]) * 12 / 7))

df_dg = pd.DataFrame({
    'Year': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'Dollars fined (from penalties of $40,000+)': [706350, 1170926, 673960, 396886, 225417, 728671, 1216165, 8558719, 18817465],
    'Penalties' : [7, 11, 7, 5, 2, 6, 9, 29, 82],
    'Employees': [105500, 113400, 121000, 129000, 135000, 143000, 158000, 163000, 170000]
})

18817465.714285713
82.28571428571429


In [90]:
df_dg['Dollars fined per employee'] = df_dg['Dollars fined per employee'] = df_dg['Dollars fined (from penalties of $40,000+)'] / df_dg['Employees']

In [91]:
df_dg

,Year,"Dollars fined (from penalties of $40,000+)",Penalties,Employees,Dollars fined per employee
0,2015,706350,7,105500,6.695261
1,2016,1170926,11,113400,10.325626
2,2017,673960,7,121000,5.569917
3,2018,396886,5,129000,3.076636
4,2019,225417,2,135000,1.669756
5,2020,728671,6,143000,5.095601
6,2021,1216165,9,158000,7.697247
7,2022,8558719,29,163000,52.507479
8,2023,18817465,82,170000,110.690971


In [92]:
#That looks fucking amazing. Let's do the same for Dollar Tree/Family Dollar and Target to compare giant retailers.

In [93]:
#OK, I manually went through the annual 10-K reports for Dollar Tree/Family Dollar and Target,
#like I did for Dollar General:

# Let’s start in 2016 because there’s no Dollar Tree SEC report for 2016 and because Target still had Canada locations until April 2015.

# https://corporate.target.com/press/releases/2015/04/target-canada-to-close-all-canadian-stores-by-apri

# Dollar Tree employees per year: (REMEMBER TO MULTIPLY BY U.S. MULTIPLIER)

# 2016: 55,300 + 112500 = 167800
# 2017: 55300 + 121500 = 176800
# 2018: 56300 + 119800 = 176100
# 2019: 57200 + 124900 = 182100
# 2020: 56900 + 136200 = 193100
# 2021: 60217 + 139110 = 199327
# 2022: 61866 + 148679 = 210565
# 2023: 65025 + 142523 = 207548


# Scrapehero indicates that there are 233 Dollar Trees in Canada. (I can't find any indication online of
#Family Dollar existing in Canada.) Scrapehero seems very likely to be accurate because its count of
#Dollar Trees and Family Dollars, in total, are very close to the figures in the company's 10-K reports:

# Scrapehero states: 8349 Family Dollars + 7927 U.S. Dollar Trees + 233 Canada Dollar Trees = 16509. 
# Dollar Tree counts 16340.

#Assuming this is roughly accurate, we can say that Dollar Tree/Family Dollar's Canada stores make up
#only 233 / 16509 of the company's locations, or 1.4%. 

#It's not exact, but this is such a small figure that it negates the need for precision, and we can likely 
#extrapolate to figure out the number of employees in the U.S.
#This means that we’re going to multiply the number of employees by 0.986 . 
#Ideally, we would get Scrapehero's figures for Canada stores per year, but the figure is so relatively small
#that I think it's okay if, for this project, I use the 0.986 multiplier.


dt_employees_list = []

dt_employees_list.append(None)
dt_employees_list.append(int(167800 * 0.986))
dt_employees_list.append(int(176800 * 0.986))
dt_employees_list.append(int(176100 * 0.986))
dt_employees_list.append(int(182100 * 0.986))
dt_employees_list.append(int(193100 * 0.986))
dt_employees_list.append(int(199327 * 0.986))
dt_employees_list.append(int(210565 * 0.986))
dt_employees_list.append(int(207548 * 0.986))

print(dt_employees_list)

# sources:
# https://corporate.dollartree.com/investors/sec-filings?form_type=10-K&year=2018
# https://www.sec.gov/edgar/browse/?CIK=935703&owner=exclude


[None, 165450, 174324, 173634, 179550, 190396, 196536, 207617, 204642]


In [94]:
#Let's update this chart:
df_dt

,Year,"Dollars fined (from penalties of $40,000+)",Penalties
0,2015,1306100,13
1,2016,487832,5
2,2017,585766,5
3,2018,2197933,17
4,2019,4388710,25
5,2020,7348248,28
6,2021,1486759,9
7,2022,4740048,21
8,2023,8515410,41


In [95]:
df_dt['Employees'] = dt_employees_list

In [96]:
df_dt['Dollars fined per employee'] = df_dt['Dollars fined (from penalties of $40,000+)'] / df_dt['Employees']

In [97]:
df_dt

,Year,"Dollars fined (from penalties of $40,000+)",Penalties,Employees,Dollars fined per employee
0,2015,1306100,13,NaN,NaN
1,2016,487832,5,165450.0,2.948516
2,2017,585766,5,174324.0,3.360214
3,2018,2197933,17,173634.0,12.658425
4,2019,4388710,25,179550.0,24.442829
5,2020,7348248,28,190396.0,38.594550
6,2021,1486759,9,196536.0,7.564818
7,2022,4740048,21,207617.0,22.830732
8,2023,8515410,41,204642.0,41.611253


In [98]:
# ==============

# HERE are TARGET'S NUMBERS. IT'S ONLY IN THE U.S. AFTER 2016:

# Target full-time, part-time and seasonal employees per year:

# 2015: 347000
# 2016: 341000
# 2017: 323000
# 2018: 345000
# 2019: 360000
# 2020: 368000
# 2021: 409000
# 2022: 450000
# 2023: 440000

tgt_employees_list = [347000, 341000, 323000, 345000, 360000, 368000, 409000, 450000, 440000]

print(tgt_employees_list)



[347000, 341000, 323000, 345000, 360000, 368000, 409000, 450000, 440000]


In [99]:
#Here's Target's dollar penalties accrued annually. Like usual, I'll have to multiply the 2023 value by 12/7.
print(df['initial_penalty_cleaned'][df.Year == 2015][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2016][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2017][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2018][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2019][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2020][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2021][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2022][df.employers_fixed == 'Target (similar names merged)'].sum())
print(df['initial_penalty_cleaned'][df.Year == 2023][df.employers_fixed == 'Target (similar names merged)'].sum())

147334 * 12 / 7 #I'll replace the 2023 value with this.

47000.0
43500.0
0.0
0.0
907915.0
261594.0
40959.0
176111.0
147334.0


252572.57142857142

In [100]:
#Let's make a DataFrame for Target now. Like usual, we'll multiply the 2023 values by 12/7 since we're in July
#for the penalties and the dollars fined.

df_tgt = pd.DataFrame({
    'Year': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'Dollars fined (from penalties of $40,000+)': [47000, 43500, 0, 0, 907915, 261594, 40959, 176111, 147334],
    'Penalties' : [1, 1, 0, 0, 9, 4, 1, 2, 1.7],
    'Employees': [347000, 341000, 323000, 345000, 360000, 368000, 409000, 450000, 440000]
})

In [101]:
df_tgt['Employees'] = tgt_employees_list

In [102]:
df_tgt['Dollars fined per employee'] = df_tgt['Dollars fined (from penalties of $40,000+)'] / df_tgt['Employees']

In [103]:
df_tgt

,Year,"Dollars fined (from penalties of $40,000+)",Penalties,Employees,Dollars fined per employee
0,2015,47000,1.0,347000,0.135447
1,2016,43500,1.0,341000,0.127566
2,2017,0,0.0,323000,0.000000
3,2018,0,0.0,345000,0.000000
4,2019,907915,9.0,360000,2.521986
5,2020,261594,4.0,368000,0.710853
6,2021,40959,1.0,409000,0.100144
7,2022,176111,2.0,450000,0.391358
8,2023,147334,1.7,440000,0.334850


In [104]:
#I now have the three charts I need to make the line graph. Thank goodness.

In [105]:
df_dg

,Year,"Dollars fined (from penalties of $40,000+)",Penalties,Employees,Dollars fined per employee
0,2015,706350,7,105500,6.695261
1,2016,1170926,11,113400,10.325626
2,2017,673960,7,121000,5.569917
3,2018,396886,5,129000,3.076636
4,2019,225417,2,135000,1.669756
5,2020,728671,6,143000,5.095601
6,2021,1216165,9,158000,7.697247
7,2022,8558719,29,163000,52.507479
8,2023,18817465,82,170000,110.690971


In [106]:
df_dt

,Year,"Dollars fined (from penalties of $40,000+)",Penalties,Employees,Dollars fined per employee
0,2015,1306100,13,NaN,NaN
1,2016,487832,5,165450.0,2.948516
2,2017,585766,5,174324.0,3.360214
3,2018,2197933,17,173634.0,12.658425
4,2019,4388710,25,179550.0,24.442829
5,2020,7348248,28,190396.0,38.594550
6,2021,1486759,9,196536.0,7.564818
7,2022,4740048,21,207617.0,22.830732
8,2023,8515410,41,204642.0,41.611253


In [107]:
df_line = pd.DataFrame({
    'Year': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'Dollar General/Dolgencorp': df_dg['Dollars fined per employee'],
    'Dollar Tree/Family Dollar': df_dt['Dollars fined per employee'],
    'Target': df_tgt['Dollars fined per employee']
})

In [108]:
df_line

,Year,Dollar General/Dolgencorp,Dollar Tree/Family Dollar,Target
0,2015,6.695261,NaN,0.135447
1,2016,10.325626,2.948516,0.127566
2,2017,5.569917,3.360214,0.000000
3,2018,3.076636,12.658425,0.000000
4,2019,1.669756,24.442829,2.521986
5,2020,5.095601,38.594550,0.710853
6,2021,7.697247,7.564818,0.100144
7,2022,52.507479,22.830732,0.391358
8,2023,110.690971,41.611253,0.334850


In [109]:
#Let's remove the year we don't want.
df_line = df_line.drop([0])

In [110]:
df_line['Dollar General/Dolgencorp'] = df_line['Dollar General/Dolgencorp'].astype(int)
df_line['Dollar Tree/Family Dollar'] = df_line['Dollar Tree/Family Dollar'].astype(int)
df_line['Target'] = df_line['Target'].astype(float).round(1)

In [111]:
df_line

,Year,Dollar General/Dolgencorp,Dollar Tree/Family Dollar,Target
1,2016,10,2,0.1
2,2017,5,3,0.0
3,2018,3,12,0.0
4,2019,1,24,2.5
5,2020,5,38,0.7
6,2021,7,7,0.1
7,2022,52,22,0.4
8,2023,110,41,0.3


In [494]:
df_line.to_csv('lede_project-3_df-for-linechart.csv')